# Sequence to Sequence Autoencoder With Tensorflow



In [ ]:
#import packages
import matplotlib.pyplot as plt 
import tensorflow as tf
import numpy as np
import scipy.io as sio

In [ ]:
#data read-test and training sets.
def read_data():
    x=sio.loadmat('vibration_nasa_1_ch5.mat')
    x=x['dataset']
    train_set_size=int(np.size(x,axis=1)*4/50)*10
    train_set=x[:,:train_set_size]
    test_set=x[:,train_set_size:]
    normalization_constant=(np.max(np.abs(train_set)))
    train_set/=normalization_constant
    test_set/=normalization_constant
    return train_set,test_set

In [ ]:
#Parameters:
time_length=200#length of each signal frame
batch_size=5#batch size for training
input_dim=1#1-d time series signal
num_units=10#number of units in LSTM cells
#Inputs & Outputs => Data
encoder_input=tf.placeholder(tf.float32,[time_length,batch_size,input_dim])#input placeholder
decoder_ground_truth=tf.placeholder(tf.float32,[time_length,batch_size,input_dim])#true output placeholder

In [ ]:
#Build encoder part
encoder_cell=tf.nn.rnn_cell.BasicLSTMCell(num_units,name='encoder',dtype=tf.float32)#LSTM cell
initial_state = encoder_cell.zero_state(batch_size, dtype=tf.float32)
encoder_outputs,encoder_state=tf.nn.dynamic_rnn(encoder_cell,encoder_input,time_major=True,
                                                initial_state=initial_state)#Run dynamic RNN
#build decoder part
decoder_cell = tf.nn.rnn_cell.BasicLSTMCell(num_units,dtype=tf.float32,name='decoder')
zero_input=tf.constant(np.zeros([time_length,batch_size,input_dim]),dtype=tf.float32)
decoder_outputs,decoder_state=tf.nn.dynamic_rnn(decoder_cell,zero_input,time_major=True,initial_state=encoder_state)
##The outputs are calculated by linear combination of the states, because the problem is a regression problem
##For this operation we need a weight kernel and a bias kernel
W2=tf.Variable(np.random.rand(num_units,input_dim),dtype=tf.float32)
b2=tf.Variable(np.random.rand(input_dim,1),dtype=tf.float32)
#output vector (It is called as vector because outputs are created as vectors of size truncated bp length)
predictions=[tf.matmul(decoder_outputs[i,:,:],W2)+b2 for i in range(decoder_outputs.get_shape().as_list()[0])]
predictions=tf.reshape(predictions,[time_length,batch_size,input_dim])


In [ ]:
error=tf.losses.mean_squared_error(decoder_ground_truth,predictions)#error

train_step=tf.train.RMSPropOptimizer(0.001).minimize(error)#optimizer (applies 1 step training)- 
                                                        #AdaGrad is preferred for its convergence rate

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())#initialize variables
    train_set,test_set=read_data()#generate time series
    k=np.reshape(train_set,[200,5,-1],order='F')
    
    data_length=np.size(k,axis=2)
    for epoch in range(100):
        for frame_id in range(data_length):
            batch_in=np.expand_dims(k[:,:,frame_id],axis=2)
            batch_out=batch_in
            [_error,_train_step,_predictions]=sess.run(
                    [error,train_step,predictions],
                    feed_dict={
                        encoder_input:batch_in,
                        decoder_ground_truth:batch_out
                    }
                )
            if frame_id%100==0:
                print(frame_id,_error)
                '''plt.figure(1)
                plt.plot(batch_in[:,1,0])
                plt.figure(2)
                plt.plot(_predictions[:,1,0])
                plt.show()'''

In [ ]:
print(k.shape)
plt.figure(1)
plt.plot(k[:,2,1])
plt.figure(2)
plt.plot(train_set[:,7])
plt.show()